In [1]:
from kamgon import *

In [2]:
class JobShopSchedulingProblem(CSPProblem):
    """
    ปัญหาการจัดตารางงานในโรงงาน (Job Shop Scheduling)
    ที่ต้องจัดสรรงานไปยังเครื่องจักรโดยคำนึงถึงลำดับการดำเนินงานและเวลาที่ใช้
    """
    
    def __init__(self, jobs: List[str], machines: List[str], 
                 operations: Dict[str, List[tuple]], 
                 processing_times: Dict[tuple, int],
                 machine_availability: Dict[str, List[tuple]]):
        """
        เริ่มต้นปัญหา Job Shop Scheduling
        
        Args:
            jobs: รายการงาน
            machines: รายการเครื่องจักร
            operations: ลำดับการดำเนินงานสำหรับแต่ละงาน [(เครื่องจักร, ลำดับ), ...]
            processing_times: เวลาที่ใช้ในการประมวลผล {(งาน, เครื่องจักร): เวลา}
            machine_availability: ช่วงเวลาที่เครื่องจักรสามารถใช้งานได้
        """
        self.jobs = jobs
        self.machines = machines
        self.operations = operations
        self.processing_times = processing_times
        self.machine_availability = machine_availability
        
        # สร้างตัวแปรสำหรับแต่ละ operation
        variables = []
        domains = {}
        
        for job in jobs:
            job_operations = operations.get(job, [])
            for i, (machine, _) in enumerate(job_operations):
                var_name = f"{job}_op{i}_{machine}"
                variables.append(var_name)
                
                # สร้าง domain ที่เป็นช่วงเวลาที่เป็นไปได้
                processing_time = processing_times.get((job, machine), 1)
                available_slots = machine_availability.get(machine, [(0, 24)])
                
                possible_start_times = []
                for start_time, end_time in available_slots:
                    for t in range(start_time, end_time - processing_time + 1):
                        possible_start_times.append(t)
                
                domains[var_name] = possible_start_times
        
        super().__init__(variables, domains)
    
    def is_consistent(self, variable: str, value: Any, assignment: Dict[str, Any]) -> bool:
        """
        ตรวจสอบข้อจำกัดของ Job Shop Scheduling
        
        Args:
            variable: ชื่อตัวแปร operation
            value: เวลาเริ่มต้นของ operation
            assignment: การจัดตารางที่มีอยู่แล้ว
            
        Returns:
            True ถ้าการจัดตารางนี้ไม่ขัดแย้งกับข้อจำกัด
        """
        job, op_index, machine = self._parse_variable(variable)
        start_time = value
        processing_time = self.processing_times.get((job, machine), 1)
        end_time = start_time + processing_time
        
        for assigned_var, assigned_start in assignment.items():
            if assigned_var == variable:
                continue
                
            assigned_job, assigned_op_index, assigned_machine = self._parse_variable(assigned_var)
            assigned_processing_time = self.processing_times.get((assigned_job, assigned_machine), 1)
            assigned_end = assigned_start + assigned_processing_time
            
            # ข้อจำกัดที่ 1: เครื่องจักรเดียวไม่สามารถทำงานหลายอย่างพร้อมกันได้
            if machine == assigned_machine:
                if not (end_time <= assigned_start or assigned_end <= start_time):
                    return False
            
            # ข้อจำกัดที่ 2: งานเดียวกันต้องทำตามลำดับที่กำหนด
            if job == assigned_job:
                if op_index < assigned_op_index and end_time > assigned_start:
                    return False
                if op_index > assigned_op_index and start_time < assigned_end:
                    return False
        
        return True
    
    def _parse_variable(self, variable: str) -> tuple:
        """แยกชื่อตัวแปรเพื่อหาข้อมูลงาน operation และเครื่องจักร"""
        parts = variable.split('_')
        job = parts[0]
        op_part = parts[1]  # เช่น 'op0', 'op1'
        op_index = int(op_part[2:])
        machine = parts[2]
        return job, op_index, machine
    
    def calculate_makespan(self, assignment: Dict[str, Any]) -> int:
        """
        คำนวณเวลารวมที่ใช้ในการทำงานทั้งหมด (makespan)
        
        Args:
            assignment: การจัดตารางที่สมบูรณ์
            
        Returns:
            เวลาสูงสุดที่ใช้ในการทำงานทั้งหมด
        """
        max_end_time = 0
        
        for variable, start_time in assignment.items():
            job, op_index, machine = self._parse_variable(variable)
            processing_time = self.processing_times.get((job, machine), 1)
            end_time = start_time + processing_time
            max_end_time = max(max_end_time, end_time)
        
        return max_end_time
    
    def get_machine_schedule(self, assignment: Dict[str, Any]) -> Dict[str, List[tuple]]:
        """
        จัดกลุ่มตารางงานตามเครื่องจักร
        
        Args:
            assignment: การจัดตารางที่สมบูรณ์
            
        Returns:
            Dictionary ที่จัดกลุ่มงานตามเครื่องจักร
        """
        machine_schedule = {machine: [] for machine in self.machines}
        
        for variable, start_time in assignment.items():
            job, op_index, machine = self._parse_variable(variable)
            processing_time = self.processing_times.get((job, machine), 1)
            end_time = start_time + processing_time
            
            machine_schedule[machine].append((job, start_time, end_time))
        
        # เรียงลำดับตามเวลาเริ่มต้น
        for machine in machine_schedule:
            machine_schedule[machine].sort(key=lambda x: x[1])
        
        return machine_schedule


In [3]:
# การใช้งาน Job Shop Scheduling Problem
if __name__ == "__main__":
    print("=== Job Shop Scheduling Problem ===")
    
    # ข้อมูลตัวอย่าง
    jobs = ['งาน1', 'งาน2', 'งาน3']
    machines = ['เครื่องA', 'เครื่องB', 'เครื่องC']
    
    # ลำดับการดำเนินงานสำหรับแต่ละงาน
    operations = {
        'งาน1': [('เครื่องA', 0), ('เครื่องB', 1), ('เครื่องC', 2)],
        'งาน2': [('เครื่องB', 0), ('เครื่องC', 1), ('เครื่องA', 2)],
        'งาน3': [('เครื่องC', 0), ('เครื่องA', 1), ('เครื่องB', 2)]
    }
    
    # เวลาที่ใช้ในการประมวลผล
    processing_times = {
        ('งาน1', 'เครื่องA'): 3,
        ('งาน1', 'เครื่องB'): 2,
        ('งาน1', 'เครื่องC'): 4,
        ('งาน2', 'เครื่องA'): 2,
        ('งาน2', 'เครื่องB'): 3,
        ('งาน2', 'เครื่องC'): 1,
        ('งาน3', 'เครื่องA'): 4,
        ('งาน3', 'เครื่องB'): 1,
        ('งาน3', 'เครื่องC'): 2
    }
    
    # ช่วงเวลาที่เครื่องจักรสามารถใช้งานได้
    machine_availability = {
        'เครื่องA': [(0, 16)],  # ใช้งานได้ 16 ชั่วโมง
        'เครื่องB': [(0, 16)],
        'เครื่องC': [(0, 16)]
    }
    
    job_shop = JobShopSchedulingProblem(
        jobs, machines, operations, processing_times, machine_availability
    )
    
    # แก้ปัญหาการจัดตารางงาน
    solution = backtracking_search(job_shop, verbose=False)
    
    if solution:
        print("การจัดตารางงานสำเร็จ:")
        
        # แสดงตารางงานของแต่ละเครื่องจักร
        machine_schedule = job_shop.get_machine_schedule(solution)
        for machine, schedule in machine_schedule.items():
            print(f"\n{machine}:")
            for job, start, end in schedule:
                print(f"  {job}: {start:2d}:00 - {end:2d}:00")
        
        # คำนวณและแสดง makespan
        makespan = job_shop.calculate_makespan(solution)
        print(f"\nMakespan (เวลารวมที่ใช้): {makespan} ชั่วโมง")
        
    else:
        print("ไม่สามารถจัดตารางงานได้ด้วยข้อจำกัดที่กำหนด")

=== Job Shop Scheduling Problem ===
การจัดตารางงานสำเร็จ:

เครื่องA:
  งาน1:  0:00 -  3:00
  งาน3:  3:00 -  7:00
  งาน2:  7:00 -  9:00

เครื่องB:
  งาน2:  0:00 -  3:00
  งาน1:  3:00 -  5:00
  งาน3:  7:00 -  8:00

เครื่องC:
  งาน3:  0:00 -  2:00
  งาน2:  3:00 -  4:00
  งาน1:  5:00 -  9:00

Makespan (เวลารวมที่ใช้): 9 ชั่วโมง
